# Load dataset and seperate into train/test/validation

In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf

batch_size = 32
training, validation, test = tfds.load('cats_vs_dogs', split=['train[:70%]', 'train[70%:80%]', 'train[90%:]'], as_supervised=True)

def preprocess(ds):
    ds = ds.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))
    ds = ds.map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y)) # converts to greyscale and zero-centers the data
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

training = preprocess(training)
validation = preprocess(validation)
test = preprocess(test)

# Set up model

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['binary_accuracy'])

# Train model

In [7]:
epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 84s 153ms/step - loss: 0.5905 - binary_accuracy: 0.6722 - val_loss: 0.5331 - val_binary_accuracy: 0.7422
Epoch 2/5
509/509 [==============================] - 79s 148ms/step - loss: 0.4642 - binary_accuracy: 0.7804 - val_loss: 0.5617 - val_binary_accuracy: 0.7349
Epoch 3/5
509/509 [==============================] - 78s 147ms/step - loss: 0.3648 - binary_accuracy: 0.8384 - val_loss: 0.4862 - val_binary_accuracy: 0.7817
Epoch 4/5
509/509 [==============================] - 78s 147ms/step - loss: 0.2431 - binary_accuracy: 0.8998 - val_loss: 0.6322 - val_binary_accuracy: 0.7894
Epoch 5/5
509/509 [==============================] - 79s 149ms/step - loss: 0.1399 - binary_accuracy: 0.9459 - val_loss: 0.6985 - val_binary_accuracy: 0.7937


# Evaluate on test set

In [8]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.6511451601982117, 0.7871882915496826]


# Train VGG19

In [9]:
import numpy as np

model = tf.keras.Sequential()
model.add(tf.keras.applications.VGG19(include_top=False, pooling='avg'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 199s 363ms/step - loss: 0.2006 - binary_accuracy: 0.9377 - val_loss: 0.0991 - val_binary_accuracy: 0.9738
Epoch 2/5
509/509 [==============================] - 173s 332ms/step - loss: 0.0612 - binary_accuracy: 0.9797 - val_loss: 0.0750 - val_binary_accuracy: 0.9802
Epoch 3/5
509/509 [==============================] - 172s 332ms/step - loss: 0.0480 - binary_accuracy: 0.9845 - val_loss: 0.0677 - val_binary_accuracy: 0.9807
Epoch 4/5
509/509 [==============================] - 172s 332ms/step - loss: 0.0378 - binary_accuracy: 0.9880 - val_loss: 0.0643 - val_binary_accuracy: 0.9828
Epoch 5/5
509/509 [==============================] - 172s 332ms/step - loss: 0.0341 - binary_accuracy: 0.9888 - val_loss: 0.0619 - val_binary_accuracy: 0.9837


# Evaluate VGG19 on test set

The VGG19 model performs much better than the regular CNN, likely due to the more robust model architecture and a larger training dataset.

In [10]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.040633171796798706, 0.9871023297309875]


# Create and train new CNN with random transformations

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.RandomFlip())
model.add(tf.keras.layers.RandomRotation((1, 1), fill_mode='nearest'))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['binary_accuracy'])

epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 80s 148ms/step - loss: 0.6145 - binary_accuracy: 0.6542 - val_loss: 0.5632 - val_binary_accuracy: 0.7026
Epoch 2/5
509/509 [==============================] - 78s 146ms/step - loss: 0.5256 - binary_accuracy: 0.7394 - val_loss: 0.5204 - val_binary_accuracy: 0.7379
Epoch 3/5
509/509 [==============================] - 79s 149ms/step - loss: 0.4789 - binary_accuracy: 0.7694 - val_loss: 0.4860 - val_binary_accuracy: 0.7688
Epoch 4/5
509/509 [==============================] - 80s 150ms/step - loss: 0.4431 - binary_accuracy: 0.7915 - val_loss: 0.4656 - val_binary_accuracy: 0.7894
Epoch 5/5
509/509 [==============================] - 80s 151ms/step - loss: 0.4067 - binary_accuracy: 0.8188 - val_loss: 0.4461 - val_binary_accuracy: 0.8006


# Evaluate on test set

Perforance is somewhat better, likely because the random transformations forced the model to learn fundemental patterns of the animals instead of learning the simple patterns present in the image.

In [13]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.4190812110900879, 0.8073946833610535]
